# Scraping WodUp

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.append('../src/')
import wodupcrawler
import os
from bokeh.models import Div, ColumnDataSource, Legend, HoverTool, CustomJS, Slider, Column
from bokeh.io import output_notebook, show, output_file, save, reset_output
import fitetl, wodupcrawler, sleepetl
import glob
import pandas as pd
from datetime import datetime
from bokeh.plotting import figure
import htmltext

output_notebook()

/Users/hasannagib/opt/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Loading BokehJS ...

In [3]:
# df = pd.read_csv(
#     '/Users/hasannagib/Documents/s3stage/fitbit/sleep.csv',
#     parse_dates=['start', 'end']
# )

df = sleepetl.read_sleep_plot_df()

In [4]:
df.head()
#df['duration'] = (df['end'] - df['start']).apply(lambda x: x.total_seconds())/60
#df.query('date == "Saturday, Sep 26, 2020"')

,date,start,end,awake,rem,light,deep,duration,time_asleep,start_hour,end_hour,7hr,8hr,9hr,7day_avg,date_str,start_time,end_time
0,2020-11-15,2020-11-14 21:36:00,2020-11-15 07:19:00,1.564333,1.982167,3.915833,2.254333,583.0,8.152333,21.60,7.32,7,8,9,8.152333,Sun Nov 15 2020,09:36 PM,07:19 AM
1,2020-11-14,2020-11-13 20:35:00,2020-11-14 05:15:00,2.183333,1.166667,4.016667,1.300000,520.0,6.483333,20.58,5.25,7,8,9,7.317833,Sat Nov 14 2020,08:35 PM,05:15 AM
2,2020-11-13,2020-11-12 21:32:00,2020-11-13 07:11:30,1.650000,1.533333,5.016667,1.450000,579.5,8.000000,21.53,7.18,7,8,9,7.545222,Fri Nov 13 2020,09:32 PM,07:11 AM
3,2020-11-12,2020-11-11 21:44:00,2020-11-12 07:00:00,1.883333,1.683333,4.166667,1.533333,556.0,7.383333,21.73,7.00,7,8,9,7.504750,Thu Nov 12 2020,09:44 PM,07:00 AM
4,2020-11-11,2020-11-10 23:00:00,2020-11-11 07:59:00,1.416667,1.550000,4.500000,1.516667,539.0,7.566667,23.00,7.98,7,8,9,7.517133,Wed Nov 11 2020,11:00 PM,07:59 AM


In [5]:
#df.to_csv('/Users/hasannagib/Documents/s3stage/fitbit/sleep.csv', index=None)

In [50]:
s = ColumnDataSource(df.query('(start_hour > 16) and (time_asleep > 0)'))

p = figure(
    plot_height=600,
    plot_width=600,
    x_range=(15,30),
    y_range=(3,10),
    tools="lasso_select, box_select, reset",
    x_axis_label='Start hour',
    y_axis_label='Time asleep (hours)',
)

p.add_layout(Legend(), 'below')
p.circle(
    x='start_hour', 
    y='time_asleep', 
    size=10,
    alpha=0.5,
    color='#3f8dff', #154ba6', #, '#3f8dff', '#7ec4ff', '#e73360'],
    source=s
)

p.add_tools(HoverTool(
        tooltips=[
            ("Duration", "@time_asleep"),
            ("7day avg", "@7day_avg"),
            ("Date", "@date_str"),
            ("Start", "@start_time"),
            ("End", "@end_time"),
        ]
    ))

s2 = ColumnDataSource(data={'x':[0, 30], 'y':[df['time_asleep'].mean(), df['time_asleep'].mean()]})
p.line(x='x', y='y', color="#154ba6", line_width=5, alpha=0.6, source=s2)

s.selected.js_on_change(
    'indices', 
    CustomJS(
        
        args={'s':s, 's2':s2},
        
        code="""
            
            const inds = s.selected.indices;
            const d = s.data;
            var ym = []
            var xm = []

            if (inds.length == 0) {
                return;
            }
            else if (inds.length == 1){
                s2.data['y'] = [d['time_asleep'][inds[0]], d['time_asleep'][inds[0]]]
            }
            else if (inds.length > 1){
                for (var i = 0; i < inds.length; i++) {
                    ym.push(d['time_asleep'][inds[i]])
                    xm.push(d['start_hour'][inds[i]])
                }

                soln = linearRegression(xm, ym)
                m = soln[0]
                b = soln[1]

                console.log(m)
                console.log(b)


                s2.data['y'] = [b, 30*m+b]
            }
            
        s.change.emit();
        s2.change.emit();
            
        """
    )
)

p.js_on_event('reset', CustomJS(
        
        args={'s':s, 's2':s2},
        code="""
        const d = s.data;
        var xm = [].slice.call(d['start_hour'])
        var ym = [].slice.call(d['time_asleep'])

        soln = linearRegression(xm, ym)
        m = soln[0]
        b = soln[1]
        
        s2.data['y'] = [b, 30*m+b]        
        s2.change.emit();
        """
))

output_dir = 'test.html'
output_file(output_dir, title="Hasan's Data Blog")
save(p, template=htmltext.bokeh_template)

'/Users/hasannagib/Desktop/projects/Wahoo-Tickrs-Logs/notebooks/test.html'

In [ ]:
# Save dashboard
output_dir = 'test.html'
output_file(output_dir, title="Hasan's Data Blog")
save(p, template=htmltext.bokeh_template)